<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/data_processing/lessons/navigacios_fajl_letoltese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RINEX navigációs fájl letöltése**

GNSS mérések utófeldolgozásához RINEX navigációs fájlokra van szükségünk. alapesetben ezeket a GNSS-vevőink rögzítik, de vannak speciális feladatok, pl. a tudományos kutatásban, amikor ezeket valamilyen adatközpontból töltjük le. Ennek egyes részleteit mutatja be ez a segédlet.

Az első megoldás, hogy a fájlt letöltjük a böngészőből, pl. az alábbi oldalról: https://igs.bkg.bund.de/searchRINEX

A második megoldás, hogy ismerjük a fájlra mutató linket és hazsnáljuk a wget parancssori programot, pl:

In [3]:
!wget https://igs.bkg.bund.de/root_ftp/EUREF/obs/2024/122/BUTE00HUN_R_20241220000_01D_RN.rnx.gz

--2024-05-08 09:59:47--  https://igs.bkg.bund.de/root_ftp/EUREF/obs/2024/122/BUTE00HUN_R_20241220000_01D_RN.rnx.gz
Resolving igs.bkg.bund.de (igs.bkg.bund.de)... 141.74.48.57
Connecting to igs.bkg.bund.de (igs.bkg.bund.de)|141.74.48.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38337 (37K) [application/x-gzip]
Saving to: ‘BUTE00HUN_R_20241220000_01D_RN.rnx.gz.1’

BUTE00HUN_R_2024122 100%[===================>]  37.44K  78.6KB/s    in 0.5s    

2024-05-08 09:59:50 (78.6 KB/s) - ‘BUTE00HUN_R_20241220000_01D_RN.rnx.gz.1’ saved [38337/38337]



A harmadik megoldás, hogy ugyanezt python orogramból tesszük meg, ehhez először telepítsük a wget python modult

In [4]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2f2b827db99e9207a304c5c237db9280b16d8635f20bb2c5aaa566080a85832b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


Majd importáljuk

In [5]:
import wget

Ezután készítsünk egy python függvényt, ami elkészíti a letöltéshez a linket. Három input adata lesz a függvénynek:

1.   http szerver neve;
2.   a könyvtárstruktúra a szerveren;
3.   az adott nap dátuma.

A köyvtárstruktúra, a fájl neve ... stb. más szervereken lehet különböző, ez itt most csak egy példa.

A példában 2024. május 1-ére vonatkozó, a BUTE állomáson rögzített adatokat töltjük le. Használjuk a python datetime modulját és a strftime függvényét, hogy a dátumot a megfelelő formátumban adhassuk meg. A fájl nevében és az elérési útvonalban az év és az év napja szerepel.

In [23]:
from datetime import date

def download_rinex_navigation(server, rpath, my_date):
  doy = my_date.strftime('%j')
  year = my_date.strftime('%Y')
  url = server + rpath + year + '/' + doy + '/' + 'BUTE00HUN_R_' + year + doy + '0000_01D_RN.rnx.gz'
  try:
    name = wget.download(url)
  except URLError:
    return None
  return name

server = 'https://igs.bkg.bund.de/'
rpath = 'root_ftp/EUREF/obs/'
my_date = date(2024, 5, 1)
print(my_date)
download_rinex_navigation(server, rpath, my_date)

2024-05-01


'BUTE00HUN_R_20241220000_01D_RN.rnx (4).gz'

Nézzük meg a letöltött fájlt!

**Vegyük automatikusan az előző napi dátumot**

Egyes esetekben, pl. tudományos célú, automatizált és rendszeres feldolgozáskor az előző napi navigációs fájlra van szükségünk.

In [24]:
from datetime import date, timedelta
yesterday = date.today() - timedelta(days=1)
print(yesterday)

2024-05-07


A letöltéshez használjuk az előbb megírt függvényt.

In [25]:
download_rinex_navigation(server, rpath, yesterday)

'BUTE00HUN_R_20241280000_01D_RN.rnx (3).gz'

**Tömörítsük ki a fájlt pythonban**

A RINEX fájlok sok helyet foglalnak, ezért rendszerint tömörítve tárolják őket. A kitömörítés sokféleképpen megoldható pythonban, itt most egy egyszerű megoldást mutatunk különösebb magyarázat nélkül.

In [26]:
import gzip
import shutil
zipped = download_rinex_navigation(server, rpath, yesterday)
unzipped = zipped[:-3]
with gzip.open(zipped, 'rb') as f_in:
    with open(unzipped, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

**További feladatok**

1.   Hibakezelés megoldása. Pl. kezelni azt a helyzetet, amikor nem létezik a fájl a szerveren
2.   Készítsünk függvényt, ami RINEX mérési fájlt tölt le és tömörít ki.

